# Mapping Cell Names to the Cell Ontology/Taxonomy

Cell Ontology (CL) (https://obofoundry.org/ontology/cl.html) is an ontology designed to classify and describe cell types across different organisms. It serves as a resource for model organism and bioinformatics databases. The ontology covers a broad range of cell types in animal cells, with over 2700 cell type classes, and provides high-level cell type classes as mapping points for cell type classes in ontologies representing other species, such as the Plant Ontology or Drosophila Anatomy Ontology. Integration with other ontologies such as Uberon, GO, CHEBI, PR, and PATO enables linking cell types to anatomical structures, biological processes, and other relevant concepts.

Cell Taxonomy (https://ngdc.cncb.ac.cn/celltaxonomy), a comprehensive and curated repository of cell types and associated cell markers encompassing a wide range of species, tissues and conditions. Combined with literature curation and data integration, the current version of Cell Taxonomy establishes a well-structured taxonomy for 3,143 cell types and houses a comprehensive collection of 26,613 associated cell markers in 257 conditions and 387 tissues across 34 species. Based on 4,299 publications and single-cell transcriptomic profiles of ∼3.5 million cells, Cell Taxonomy features multifaceted characterization for cell types and cell markers, involving quality assessment of cell markers and cell clusters, cross-species comparison, cell composition of tissues and cellular similarity based on markers.

Here we provide several functions that convert the cell names you annotated to their corresponding Cell Ontology/Taxonomy names and IDs.

All analysis are performed with `omicverse.single.CellOntologyMapper` class.

- CellOntologyMapper: Zeng, Z., Wang, X., & Du, H. (2025). CellOntologyMapper: Consensus mapping of cell type annotation. bioRxiv, 2025-06.

- Cell Ontology: Mungall, C., Haendel, M., Dahdul, W., Ibrahim, N., Segerdell, E., Blackburn, D., Comte, A., Niknejad, A., and Decechi, A. (2016), Uberon Ontology, http://purl.obolibrary.org/obo/uberon/releases/2016-01-26/uberon.owl
- Cell Taxonomy: Jiang, S., Qian, Q., Zhu, T., Zong, W., Shang, Y., Jin, T., ... & Zhang, Z. (2023). Cell Taxonomy: a curated repository of cell types with multifaceted characterization. Nucleic Acids Research, 51(D1), D853-D860.


<div class="admonition warning">
  <p class="admonition-title">Note</p>
  <p>
    Support for Cell Taxonomy is available in version 1.7.2rc1. You can install it with 
      
      pip install git+https://github.com/Starlitnightly/omicverse.git
      
    or 
    
      pip install omicverse==1.7.2rc1.
      
  </p>
</div>

In [1]:
import scanpy as sc
#import pertpy as pt
import omicverse as ov
ov.plot_set()

%load_ext autoreload
%autoreload 2
!date

🔬 Starting plot initialization...
🧬 Detecting CUDA devices…
✅ [GPU 0] NVIDIA GeForce RTX 2080 Ti
    • Total memory: 10.7 GB
    • Compute capability: 7.5
✅ [GPU 1] NVIDIA GeForce RTX 2080 Ti
    • Total memory: 10.7 GB
    • Compute capability: 7.5

   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

🔖 Version: 1.7.6rc1   📚 Tutorials: https://omicverse.readthedocs.io/
✅ plot_set complete.

2025年 08月 05日 星期二 11:28:00 CST


## Prepared datasets

Before you convert you cell names, you need to annotate at first. Here, we used haber_2017_regions dataset from `pertpy` to test.

In [2]:
import pertpy as pt
adata = pt.dt.haber_2017_regions()
adata

AnnData object with n_obs × n_vars = 9842 × 15215
    obs: 'batch', 'barcode', 'condition', 'cell_label'

In [3]:
adata.obs['cell_label'].unique()

['Enterocyte.Progenitor', 'Stem', 'TA.Early', 'TA', 'Tuft', 'Enterocyte', 'Goblet', 'Endocrine']
Categories (8, object): ['Endocrine', 'Enterocyte', 'Enterocyte.Progenitor', 'Goblet', 'Stem', 'TA', 'TA.Early', 'Tuft']

## Download the CL Model

Before we start our analysis, we need to download the `cl.json` from Cell Ontology.

```shell
# Download cl.ono fro OBO page.
!mkdir new_ontology
!wget http://purl.obolibrary.org/obo/cl/cl.json -O new_ontology/cl.json
```

But we have also provided a function named `omicverse.single.download_cl()` to download it automatically. The benefit of this function is that it can choose an appropriate source to download the file even if you encounter a network error.

There are some alternative links to download the file manual. :

- Google Drive: https://drive.google.com/uc?export=download&id=1niokr5INjWFVjiXHfdCoWioh0ZEYCPkv
- Lanzou(蓝奏云): https://www.lanzoup.com/iN6CX2ybh48h

In [4]:
ov.single.download_cl(output_dir="new_ontology", filename="cl.json")


[1/3] Trying Official OBO Library...
    URL: http://purl.obolibrary.org/obo/cl/cl.json
    Description: Direct download from official Cell Ontology
    → Downloading...


    → Downloaded: 35.25 MB
    ✓ File validation successful
    ✓ Successfully downloaded from Official OBO Library
    File saved to: new_ontology/cl.json
    File size: 35.25 MB


(True, 'new_ontology/cl.json')

## Prepare the CellOntologyMapper

Because the CellOntologyMapper rely on the NLP embedding model of SentenceTransformer. So we need to choose a NLP embedding from huggingface. Here are some recommendation. 

You can found the entire list from [huggingface](https://huggingface.co/models?library=sentence-transformers) or [huggingface mirror](https://hf-mirror.com/models?library=sentence-transformers).

- BAAI/bge-base-en-v1.5
- BAAI/bge-small-en-v1.5
- sentence-transformers/all-MiniLM-L6-v2
- ...

In [6]:
# !huggingface-cli download sentence-transformers/all-MiniLM-L6-v2 --local-dir my_models/sentence-transformers/all-MiniLM-L6-v2
mapper = ov.single.CellOntologyMapper(
    cl_obo_file="new_ontology/cl.json",
    # model_name="my_models/sentence-transformers/all-MiniLM-L6-v2",
    model_name="my_models/BAAI/bge-base-en-v1.5",
    # local_model_dir="./my_models"
)

🔨 Creating ontology resources from OBO file...
📖 Parsing ontology file...
🧠 Creating NLP embeddings...
🔄 Loading model my_models/BAAI/bge-base-en-v1.5...
📁 Loading local model from: my_models/BAAI/bge-base-en-v1.5
✓ Local model loaded successfully!
🔄 Encoding 18426 ontology labels...


Batches:   0%|          | 0/576 [00:00<?, ?it/s]

💾 Embeddings saved to: new_ontology/ontology_embeddings.pkl
📋 Ontology mappings included: 18426 cell types
✓ Ontology resources creation completed!


If you have calculated the embedding of cell ontology, you can load it directly.

In [7]:
mapper = ov.single.CellOntologyMapper(
    cl_obo_file="new_ontology/cl.json",
    embeddings_path='new_ontology/ontology_embeddings.pkl',
    local_model_dir="./my_models"
)

📥 Loading existing ontology embeddings...
📥 Loaded embeddings for 18426 ontology labels
📋 Ontology mappings loaded: 18426 cell types


## Mapping Celltype

We can use `map_adata` to calculate simility between the cell name in our datasets and cell name in cell ontology.

In [8]:
mapping_results = mapper.map_adata(
    adata, 
    cell_name_col='cell_label'
)

📊 Using 8 unique cell names from column 'cell_label'
🔄 Loading model my_models/BAAI/bge-base-en-v1.5...
📁 Loading local model from: my_models/BAAI/bge-base-en-v1.5
✓ Local model loaded successfully!
🎯 Mapping 8 cell names...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📝 Applying mapping results to AnnData...
✓ Mapping completed: 8/8 cell names have high confidence mapping


In [9]:
# ov.pl.embedding(
#    adata,
#    basis='X_umap',
#    color=['cell_label','cell_ontology','cell_ontology_cl_id','cell_ontology_ontology_id'],
#    wspace=0.55,
#    ncols=2
# )

In [10]:
mapper.print_mapping_summary(mapping_results, top_n=15)


MAPPING STATISTICS SUMMARY
Total mappings:		8
High confidence:	8 (100.00%)
Low confidence:		0
Average similarity:	0.747
Median similarity:	0.750

TOP 15 MAPPING RESULTS
------------------------------------------------------------
✓ Enterocyte -> enterocyte (Similarity: 0.855)
✓ Enterocyte.Progenitor -> intestinal enteroendocrine progenitor (Similarity: 0.810)
✓ Endocrine -> endocrine process (Similarity: 0.781)
✓ Stem -> Brain stem (Similarity: 0.766)
✓ Tuft -> tuft cell of anorectum (Similarity: 0.735)
✓ TA -> Ttr (Mmus) (Similarity: 0.701)
✓ Goblet -> anorectum goblet cell (Similarity: 0.693)
✓ TA.Early -> SST (Similarity: 0.637)


## Mapping Cell Types with LLM Assistance

In addition, we often use abbreviations to name our cell types, such as `TA` and `TA.Early` in our data sets. Calculating similarity to the Cell Ontology directly can be confusing, so we use an LLM to expand these abbreviated cell names.

To do so, specify the following arguments:

* **api\_type**: `openai`, `anthropic`, `ollama`, or any other API that follows the OpenAI format
* **tissue\_context**: the tissue source of the single-cell data set
* **species**: the species from which the data set was derived
* **study\_context**: any additional information that may help the model expand the cell-type name
* **api_key**: the apikey of your model. 



In [11]:
mapper.setup_llm_expansion(
    api_type="openai", model='gpt-4o-2024-11-20',
    tissue_context="gut",    # 组织上下文
    species="mouse",                   # 物种信息
    study_context="Epithelial cells from the small intestine and organoids of mice. Some of the cells were also subject to Salmonella or Heligmosomoides polygyrus infection",
    api_key="sk-*"
)


✓ Loaded 0 cached abbreviation expansions
✗ Missing required library: No module named 'openai'
📦 Install required packages based on your API type:
   - OpenAI: pip install openai
   - Anthropic: pip install anthropic
   - 通义千问: pip install dashscope
   - 文心一言: pip install ernie-bot-sdk
   - 智谱GLM: pip install zhipuai
   - Ollama/Spark/Doubao: pip install requests


False

You can choose any other model api from the alternative provider, such as `ohmygpt`. But the format of openai should observe the rule of openai.

In [12]:
mapper.setup_llm_expansion(
    api_type="custom_openai",
    api_key="sk-*",
    model="gpt-4.1-2025-04-14",
    base_url="https://api.ohmygpt.com/v1"
)

✓ Loaded 0 cached abbreviation expansions
✗ Missing required library: No module named 'openai'
📦 Install required packages based on your API type:
   - OpenAI: pip install openai
   - Anthropic: pip install anthropic
   - 通义千问: pip install dashscope
   - 文心一言: pip install ernie-bot-sdk
   - 智谱GLM: pip install zhipuai
   - Ollama/Spark/Doubao: pip install requests


False

In [13]:
mapping_results = mapper.map_adata_with_expansion(
    adata=adata,
    cell_name_col='cell_label',
    threshold=0.5,
    expand_abbreviations=True  # 启用缩写扩展
)
mapper.print_mapping_summary(mapping_results, top_n=15)

📊 Using 8 unique cell names from column 'cell_label'
⚠️  Abbreviation expansion requested but LLM client not configured
💡 To enable LLM expansion, use: mapper.setup_llm_expansion()
🎯 Performing direct ontology mapping with LLM selection
🎯 Mapping 8 cell names...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📝 Applying mapping results to AnnData...
✓ Mapping completed:
  📊 8/8 cell names have high confidence mapping
  🔄 0/8 cell names underwent abbreviation expansion

MAPPING STATISTICS SUMMARY
Total mappings:		8
High confidence:	8 (100.00%)
Low confidence:		0
Average similarity:	0.747
Median similarity:	0.750

TOP 15 MAPPING RESULTS
------------------------------------------------------------
✓ Enterocyte -> enterocyte (Similarity: 0.855)
✓ Enterocyte.Progenitor -> intestinal enteroendocrine progenitor (Similarity: 0.810)
✓ Endocrine -> endocrine process (Similarity: 0.781)
✓ Stem -> Brain stem (Similarity: 0.766)
✓ Tuft -> tuft cell of anorectum (Similarity: 0.735)
✓ TA -> Ttr (Mmus) (Similarity: 0.701)
✓ Goblet -> anorectum goblet cell (Similarity: 0.693)
✓ TA.Early -> SST (Similarity: 0.637)


In [14]:
#ov.pl.embedding(
#    adata,
#    basis='X_umap',
#    color=['cell_label','cell_ontology','cell_ontology_cl_id','cell_ontology_ontology_id'],
#    wspace=0.55,
#    ncols=2
#)

We can now see that `TA` and `TA.Early` map successfully to `transit amplifying cell`.

## Cell Taxonomy

To enhance the annotation of cell type, we can choose not only from Cell Ontology, we can also use `Cell Taxonomy` database as another reference.

In [15]:
# !wget https://download.cncb.ac.cn/celltaxonomy/Cell_Taxonomy_resource.txt -O new_ontology/Cell_Taxonomy_resource.txt

In [16]:
mapper.load_cell_taxonomy_resource("new_ontology/Cell_Taxonomy_resource.txt", 
                                   species_filter=["Homo sapiens", "Mus musculus"])

📊 Loading Cell Taxonomy resource from: new_ontology/Cell_Taxonomy_resource.txt
✓ Loaded 226222 taxonomy entries
🐭 Filtered by species ['Homo sapiens', 'Mus musculus']: 224736/226222 entries
🧠 Creating embeddings for 2540 taxonomy cell types...


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

✓ Created taxonomy embeddings for 2540 cell types
📈 Species distribution:
  🐭 Mus musculus: 141727 entries
  🐭 Homo sapiens: 83009 entries
🧬 Unique cell types: 2540
🎯 Unique markers: 25818


True

Similiarly, we can use `map_adata_with_taxonomy` to perform the mapping.

In [17]:
enhanced_results = mapper.map_adata_with_taxonomy(
    adata, 
    cell_name_col="cell_label",
    new_col_name="enhanced_cell_ontology",
    expand_abbreviations=True,
    use_taxonomy=True,
    species="Mus musculus",
    tissue_context="Gut",
    threshold=0.3,
)

📊 Using 8 unique cell names from column 'cell_label'
⚠️  Abbreviation expansion requested but LLM client not configured
💡 To enable LLM expansion, use: mapper.setup_llm_expansion()
🎯 Performing direct ontology mapping with LLM selection
🎯 Mapping 8 cell names...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📊 Enhancing with taxonomy resource using NLP similarity...
🧬 Taxonomy enhancement: Processing 8 cell names...
      ✓ Taxonomy match: 'Enterocyte.Progenitor' → 'Enterocyte progenitor cell' (Homo sapiens) (sim: 0.922)
      ✓ Taxonomy match: 'intestinal enteroendocrine progenitor' → 'Enteroendocrine progenitor cell' (Mus musculus) (sim: 0.951)
      ✓ Taxonomy match: 'Stem' → 'Stem cell' (Mus musculus) (sim: 0.710)
      ✓ Taxonomy match: 'Brain stem' → 'Brainstem motor neuron' (Mus musculus) (sim: 0.733)
      ✓ Taxonomy match: 'TA.Early' → 'Early T lineage precursor' (Mus musculus) (sim: 0.688)
      ✓ Taxonomy match: 'SST' → 'Sst GABAergic cortical interneuron' (Mus musculus) (sim: 0.743)
      ✓ Taxonomy match: 'TA' → 'Transit amplifying cell' (Mus musculus) (sim: 0.636)
      ✓ Taxonomy match: 'Tuft' → 'Intestinal tuft cell' (Mus musculus) (sim: 0.769)
      ✓ Taxonomy match: 'tuft cell of anorectum' → 'Tuft cell of colon' (Homo sapiens) (sim: 0.896)
      ✓ Taxonomy match: 'Enter

In [18]:
mapper.print_mapping_summary_taxonomy(enhanced_results)


ENHANCED MAPPING SUMMARY (ONTOLOGY + TAXONOMY)
Total mappings:		8
High confidence:	8 (100.00%)
Low confidence:		0
Average similarity:	0.747
LLM expansions:		0
Taxonomy enhanced:	8

DETAILED MAPPING RESULTS (Top 10)
--------------------------------------------------------------------------------

 1. [✓] Enterocyte
     🎯 Ontology: enterocyte
        Similarity: 0.855
        CL ID: CL:0000584
     🧬 Taxonomy: Enterocyte
        Similarity: 1.000
        Matched from: Enterocyte
        CT ID: CT:00000594
        🎯 Marker: Btnl1
        🧬 Gene: Btn,Btnl3,Gm316,Gm33,NG1
        🆔 ENTREZ: 100038862


 2. [✓] Enterocyte.Progenitor
     🎯 Ontology: intestinal enteroendocrine progenitor
        Similarity: 0.810
        CL ID: CL:4052006
     🧬 Taxonomy: Enteroendocrine progenitor cell
        Similarity: 0.951
        Matched from: intestinal enteroendocrine progenitor
        CT ID: CT:00002674
        🎯 Marker: Neurog3
        🧬 Gene: Ato,Atoh5,Math4,Math4B,bHLH,bHLHa7,ng,ngn3,NEUROG3
  

In [19]:
#ov.pl.embedding(
#    adata,
#    basis='X_umap',
#    color=['cell_label','cell_ontology','cell_ontology_cl_id','cell_ontology_ontology_id',
#          'enhanced_cell_ontology_taxonomy_match','enhanced_cell_ontology_ct_id'],
#    wspace=0.55,
#    ncols=2
#)

## Mapping Check

We can also query the matching result manually.

In [20]:
res=mapper.find_similar_cells("T helper cell", top_k=10)
res=mapper.find_similar_cells("Macrophage", top_k=8)


🎯 Ontology cell types most similar to 'T helper cell':
 1. helper T cell                            (Similarity: 0.840)
 2. CD4-positive helper T cell               (Similarity: 0.783)
 3. T-helper 9 cell                          (Similarity: 0.776)
 4. T cell domain                            (Similarity: 0.760)
 5. T-helper 2 cell activation               (Similarity: 0.755)
 6. T-helper 1 cell activation               (Similarity: 0.754)
 7. T-helper cell differentiation            (Similarity: 0.753)
 8. T-helper 1 cell                          (Similarity: 0.752)
 9. T-helper 2 cell chemotaxis               (Similarity: 0.752)
10. regulation of T-helper 2 cell activation (Similarity: 0.747)

🎯 Ontology cell types most similar to 'Macrophage':
 1. cycling macrophage                       (Similarity: 0.834)
 2. pleural macrophage                       (Similarity: 0.824)
 3. macrophage                               (Similarity: 0.816)
 4. central nervous system macrophage        (

In [21]:
# Enterocyte.Progenitor
res=mapper.find_similar_cells_taxonomy("T helper cell", top_k=2)


🧬 Taxonomy cell types most similar to 'T helper cell':
 1. Helper T cell                            (Similarity: 0.921)
     🐭 Species: Mus musculus
     🎯 Marker: Tigit
     🆔 CT ID: CT:00000919

 2. T-helper 2 cell                          (Similarity: 0.892)
     🐭 Species: Homo sapiens
     🎯 Marker: BATF
     🆔 CT ID: CT:00000503



In [22]:
mapper.get_cell_info("regulatory T cell")


ℹ️  === regulatory T cell ===
🆔 Ontology ID: http://purl.obolibrary.org/obo/CL_0000815
📝 Description: regulatory T cell: A T cell which regulates overall immune responses as well as the responses of other T cell subsets through direct cell-cell contact and cytokine release. This cell type may express FoxP3 and CD25 and secretes IL-10 and TGF-beta.


{'name': 'regulatory T cell',
 'description': 'regulatory T cell: A T cell which regulates overall immune responses as well as the responses of other T cell subsets through direct cell-cell contact and cytokine release. This cell type may express FoxP3 and CD25 and secretes IL-10 and TGF-beta.',
 'ontology_id': 'http://purl.obolibrary.org/obo/CL_0000815'}

In [23]:
# 获取详细的taxonomy信息
info_list = mapper.get_cell_info_taxonomy(
    "Helper T cell", 
    species="Mus musculus"
)


🧬 === Cell Taxonomy Information: Helper T cell ===

📊 Entry 1:
🐭 Species: Mus musculus
🆔 CT ID: CT:00000919
🎯 Cell Marker: Tigit
🔗 Cell Ontology ID: CL:0000912
🧬 Gene Information:
   Gene Alias: Vst,Vstm3
   ENTREZ ID: 100043314
   Ensembl ID: ENSMUSG00000071552
   UniProt: A0A0B4J1G6
   PFAM: PF07686
   GO Terms: GO:0005102,GO:0005102,GO:0005102,GO:0005515,GO:0009986,GO:0009986,GO:0009986,GO:0032695,GO:0032695,G...
🧬 Tissue Ontology ID: nan
📚 PMID: 34548483
📄 Source: OnClass

📊 Entry 2:
🐭 Species: Mus musculus
🆔 CT ID: CT:00000919
🎯 Cell Marker: Klrb1
🔗 Cell Ontology ID: CL:0000912
🧬 Gene Information:
   Gene Alias: 4930431A04Rik,Gm4696,Klrb1g,Klrb6,Ly-5,Ly-55,Ly5,Ly55,NKR-P1G,Nkr,Nkrp-1e,Nkrp1g
   ENTREZ ID: 100043861
   Ensembl ID: ENSMUSG00000079299
   UniProt: A0A1U9W1A8,Q0ZUP1
   PFAM: PF00059
   GO Terms: GO:0005515,GO:0016020,GO:0016021,GO:0030246,GO:0038023
🧬 Tissue Ontology ID: UBERON:0002405
📚 PMID: nan
📄 Source: PanglaoDB

📊 Entry 3:
🐭 Species: Mus musculus
🆔 CT ID: CT:000

In [24]:
mapper.get_cell_info("regulatory T cells")

✗ Cell type not found: regulatory T cells
🔍 Found 0 cell types containing 'regulatory t cells':


In [25]:
my_categories = ["immune cell", "epithelial"]
mapper.browse_ontology_by_category(categories=my_categories, max_per_category=5)

📂 === Browse Ontology Cell Types by Category ===

🔍 Found 0 cell types containing 'immune cell':
--------------------------------------------------
🔍 Found 398 cell types containing 'epithelial':
  1. NS forest marker set of airway submucosal gland collecting duct epithelial cell (Human lung).
  2. epithelial fate stem cell
  3. epithelial cell
  4. ciliated epithelial cell
  5. duct epithelial cell
... 393 more results

🏷️  【epithelial related】 (Showing top 5):
  1. NS forest marker set of airway submucosal gland collecting duct epithelial cell (Human lung).
  2. epithelial fate stem cell
  3. epithelial cell
  4. ciliated epithelial cell
  5. duct epithelial cell
--------------------------------------------------


In [26]:
# 查看前50个细胞类型
res=mapper.list_ontology_cells(max_display=10)


📊 Total 18426 cell types in ontology

📋 First 10 cell types:
  1. TAC1
  2. STAB1
  3. TLL1
  4. MSR1
  5. TNC
  6. ROS1
  7. TNIP3
  8. HOMER3
  9. FCGR2B
 10. BPIFB2
... 18416 more cell types
💡 Use return_all=True to get complete list


In [27]:
# 了解本体论的整体情况
stats = mapper.get_ontology_statistics()

📊 === Ontology Statistics ===
📝 Total cell types: 18426
📏 Average name length: 31.4 characters
📏 Shortest name length: 3 characters
📏 Longest name length: 144 characters

🔤 Most common words:
  of: 5647 times
  cell: 3955 times
  regulation: 3246 times
  negative: 1030 times
  positive: 1026 times
  process: 990 times
  development: 883 times
  differentiation: 730 times
  muscle: 675 times
  in: 594 times
